In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
data_dir = "animals"
N_EPOCHS = 50
BATCH_SIZE  =128
NUM_CLASSES = 90
#NUM_WORKERS = 4  # Number of dataloader workers

In [ ]:
model = models.resnet50(weights = models.ResNet50_Weights.DEFAULT).to(device)     # importing resnet50 from torchvision.models

fc_features = model.fc.in_features      # no. of input features to the fully connected layer of resnet50

model.fc = nn.Sequential(nn.Linear(fc_features, NUM_CLASSES),       # changing the fully connected layer to output 90 classes instead of 1000 classses.
                         nn.ReLU(), torch.nn.Softmax()).to(device)
# freezing the inital layer parameters
for param in model.parameters():
    param.requires_grad = False
# unfreezing the parameters last convulution and fully connected layer
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
summary(model,input_size = ( 3,224,224))

In [ ]:
# loading dataset and applying resizing the images to (3,224,224) as resnet requires images of sizes (224,224) as inputs.

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_val_transforms = transforms.Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_dataset = torchvision.datasets.ImageFolder(
    root=data_dir)
len(img_dataset)

In [ ]:
# SPLITTING DATASET INTO TRAIN,TEST AND VAL.
# train_dataset,val_dataset,test_dataset = random_split(img_dataset,[int(0.6*(len(img_dataset))),int(0.2*(len(img_dataset))),int(0.2*(len(img_dataset)))])
train_dataset, val_dataset, test_dataset = random_split(
    img_dataset, [3240, 1080, 1080])

train_dataset.dataset.transform = train_transforms
test_dataset.dataset.transform = test_val_transforms
val_dataset.dataset.transform = test_val_transforms

In [ ]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
model = model.to(device)

train_dataloader = DataLoader(
    train_dataset, batch_size = BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(
    val_dataset, batch_size = len(val_dataset))
test_dataloader = DataLoader(
    test_dataset, batch_size = len(test_dataset))

In [ ]:
def f1_score(preds,labels):
    preds  = torch.argmax(preds, dim = 1)
   
 
    tp = torch.zeros(NUM_CLASSES)
    fp = torch.zeros(NUM_CLASSES)
    fn = torch.zeros(NUM_CLASSES)

    for i in range(NUM_CLASSES):
        a = torch.where(preds == i, preds, i)
        b = torch.where(labels == i, labels, i)
        c = torch.where(preds != i, preds, i)
        d = torch.where(labels != i, labels, i)

        tp[i] = torch.where(a == b, a, b).sum().item()
        fp[i] = torch.where(a == d, a, d).sum().item()
        fn[i] = torch.where(c == b, c, b).sum().item()

    precision = tp/(tp + fp + 1e-5)
    recall = tp/(tp + fn + 1e-5)

    f1_scores = 2*(precision*recall)/(precision + recall + 1e-5)

    return f1_scores.mean().item()

In [ ]:
def accuracy_fn(preds, labels):

    acc = (preds.argmax(dim = 1) == labels).sum().item()
    return acc/len(labels)

In [ ]:
train_losses = []
val_losses = []
train_accuracy, val_accuracy = [], []
best_model = model
best_val_loss = float("inf")


for epoch in tqdm(range(N_EPOCHS)):
    # training loop
    train_loss, val_loss = 0, 0
    train_F1_score, val_F1_score = 0, 0 
    train_acc, val_acc = 0, 0 
    

    for (X, y) in (train_dataloader):
        X, y = X.to(device), y.to(device)
        model.train()
        y_pred = model(X.detach())
        train_loss = loss_fn(y_pred, y)
        train_acc = accuracy_fn(y_pred, y)
        train_F1_score = f1_score(y_pred, y)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # validation loop
    with torch.inference_mode():
        for (X, y) in tqdm(val_dataloader):
            X, y = X.to(device), y.to(device)
            model.eval()
            y_pred = model(X.detach())
            val_loss = loss_fn(y_pred, y)
            val_acc = accuracy_fn(y_pred, y)
            val_F1_score = f1_score(y_pred, y)


    train_accuracy.append(train_acc)
    val_accuracy.append(val_acc)
    train_losses.append(train_loss)
    val_losses.append(val_loss)


    print(f"EPOCH {epoch+1}: training_loss: {train_loss:.4f}, validation_loss: {val_loss:.4f}")
    print(f"EPOCH {epoch+1}: training_accuracy: {train_acc:.4f}, validation_accuracy: {val_acc}")
    print(f"EPOCH {epoch+1}: training_F1 Score: {train_F1_score:.4f}, validation_F1 Score: {val_F1_score}")

In [ ]:
with torch.inference_mode():
        for (X, y) in tqdm(test_dataloader):
            X, y = X.to(device), y.to(device)
            model.eval()
            y_pred = model(X.detach())
            test_loss = loss_fn(y_pred, y)
            test_accuracy = accuracy_fn(y_pred, y)
            test_F1_score = f1_score(y_pred, y)

print(f"test loss :{test_loss :.4f},test accuracy :{test_accuracy :.4f}, test F1 Score :{test_F1_score :.4f}")

In [ ]:
save_path = os.path.join(path,"BESTMODEL.pth")
torch.save(model.state_dict(),save_path)